In [1]:
# Load extensions
%load_ext autoreload
%autoreload 2

In [2]:
# Importeren van bibliotheken
import math
import numpy as np
from sympy import *
import matplotlib.pyplot as plt

from berekeningen_andreas import Brug

Berekeningen gaan uit van volgende situatie:
![Berekeningen](figures/berekeningen_andreas.svg)

In [3]:
brug = Brug(1, 1, 0.1, 0.1, 3)
brug.__dict__

{'h1': 1,
 'h2': 1,
 'd1': 0.1,
 'd2': 0.1,
 'N': 3,
 'cosa': 0.8320502943378437,
 'sina': 0.554700196225229,
 'cosb': 0.8320502943378437,
 'sinb': 0.554700196225229}

In [4]:
A, B = brug.calc_reactie_krachten()
A, B

([[1, 1, 0, 0, 0, 0, 0, 0],
  [0, 0, 1, 1, 0, 0, 0, 0],
  [0, 0, 0, 0, 1, 1, 0, 0],
  [0, 0, 0, 0, 0, 0, 1, 1],
  [0, 0.5, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0.5, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0.5, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0.5]],
 [400.0, 800.0, 400.0, 400.0, 400.0, 466.66666666666663, 400.0, 400.0])

In [6]:
s = np.linalg.solve(A,B)
{
    'Va': s[0],
    'F1': s[1] + s[2],
    'F2': s[3] + s[4],
    'F3': s[5] + s[6],
    'Vb': s[7],
}, s

({'Va': -400.0,
  'F1': 666.6666666666667,
  'F2': 533.3333333333333,
  'F3': 400.0,
  'Vb': 800.0},
 array([-400.        ,  800.        , -133.33333333,  933.33333333,
        -400.        ,  800.        , -400.        ,  800.        ]))